https://www.unitstatistics.com/starcraft2/ used for getting base unit Hp, Dps etc.

Based on the data the following information was used for the units:

| Name      | Race  | Type                | HP         | Speed         | Sight | Attack name       | G. Attack | G. DPS     | A. DPS     | Cooldown         | Range |
|-----------|-------|---------------------|------------|---------------|-------|-------------------|-----------|------------|------------|------------------|-------|
| Zergling  | Zerg  | Biological, Light  | 35         | 4.13 (+2.45)  | 8     | Claws             | 5 (+1)    | 10 (+2)    | -          | 0.497 (-0.143)   | 0.1   |
| Marine    | Terran| Biological, Light  | 45 (+10)   | 3.15 (+1.57)  | 9     | C-14 Gauss Rifle  | 6 (+1)    | 9.8 (+1.6) | 9.8 (+1.6) | 0.61 (-0.203)    | 5     |




In [22]:
from IPython.display import Markdown

table = """
| Name      | Race  | Type                | HP         | Speed         | Sight | Attack name       | G. Attack | G. DPS     | A. DPS     | Cooldown         | Range |
|-----------|-------|---------------------|------------|---------------|-------|-------------------|-----------|------------|------------|------------------|-------|
| Zergling  | Zerg  | Biological, Light  | 35         | 4.13 (+2.45)  | 8     | Claws             | 5 (+1)    | 10 (+2)    | -          | 0.497 (-0.143)   | 0.1   |
| Marine    | Terran| Biological, Light  | 45 (+10)   | 3.15 (+1.57)  | 9     | C-14 Gauss Rifle  | 6 (+1)    | 9.8 (+1.6) | 9.8 (+1.6) | 0.61 (-0.203)    | 5     |
"""

display(Markdown(table))


| Name      | Race  | Type                | HP         | Speed         | Sight | Attack name       | G. Attack | G. DPS     | A. DPS     | Cooldown         | Range |
|-----------|-------|---------------------|------------|---------------|-------|-------------------|-----------|------------|------------|------------------|-------|
| Zergling  | Zerg  | Biological, Light  | 35         | 4.13 (+2.45)  | 8     | Claws             | 5 (+1)    | 10 (+2)    | -          | 0.497 (-0.143)   | 0.1   |
| Marine    | Terran| Biological, Light  | 45 (+10)   | 3.15 (+1.57)  | 9     | C-14 Gauss Rifle  | 6 (+1)    | 9.8 (+1.6) | 9.8 (+1.6) | 0.61 (-0.203)    | 5     |


In [23]:
import copy

In [24]:
class CurrentBoard:
    def __init__(self, width=30, height=10, marine_count=4, zergling_count=6):
        self.width = width
        self.height = height
        self.board = [[None for _ in range(width)] for _ in range(height)]
        self.marines = []  # List to store Marine objects
        self.zerglings = []  # List to store Zergling objects
        self.center_x = width // 2
        self.center_y = height // 2
        self.marine_count = marine_count
        self.zergling_count = zergling_count

    def is_in_melee_combat(self, unit):
        # Check if unit is adjacent to an enemy unit
        enemy_team = 'Zergling' if unit.team == 'Marine' else 'Marine'

        # Check adjacent squares (orthogonal directions)
        directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]

        for dx, dy in directions:
            check_x = unit.x + dx
            check_y = unit.y + dy

            # Check if the adjacent square is within board bounds
            if (0 <= check_x < self.width and
                    0 <= check_y < self.height):
                adjacent_unit = self.board[check_y][check_x]

                # If there's an enemy unit in an adjacent square
                if (adjacent_unit is not None and
                        adjacent_unit.team == enemy_team):
                    return True

        return False

    def get_attackable_targets(self, unit):
        targets = []
        enemy_team = 'Zergling' if unit.team == 'Marine' else 'Marine'

        # Determine targets based on unit type and attack range
        enemy_list = self.zerglings if unit.team == 'Marine' else self.marines

        for potential_target in enemy_list:
            # Calculate Manhattan distance
            distance = abs(unit.x - potential_target.x) + abs(unit.y - potential_target.y)

            # Marine attacks at range, Zerglings attack in melee
            if unit.team == 'Marine':
                if distance <= unit.attack_range:
                    targets.append(potential_target)
            else:  # Zerglings
                if distance == 1:  # Melee range
                    targets.append(potential_target)

        return targets

    def get_possible_actions(self, unit):
        actions = []

        # Check if unit is in melee combat
        if self.is_in_melee_combat(unit):
            # Can only attack, no movement allowed
            return [('attack', target) for target in self.get_attackable_targets(unit)]

        # If not in melee, can either move or attack
        possible_moves = self.get_possible_moves(unit)
        attack_targets = self.get_attackable_targets(unit)

        # Add move actions
        for move in possible_moves:
            actions.append(('move', move))

        # Add attack actions
        for target in attack_targets:
            actions.append(('attack', target))

        return actions


    def get_possible_moves(self, unit):
        """
        Generate all possible moves for a unit
        Moves are full speed in any of 8 directions
        """
        moves = []

        # 8 possible movement directions (including diagonals)
        directions = [
            (0, 1),  # Up
            (0, -1),  # Down
            (1, 0),  # Right
            (-1, 0),  # Left
            (1, 1),  # Up-Right
            (1, -1),  # Down-Right
            (-1, 1),  # Up-Left
            (-1, -1)  # Down-Left
        ]

        for dx, dy in directions:
            # Calculate end position based on unit's full speed
            move_x = unit.x + dx * unit.speed
            move_y = unit.y + dy * unit.speed

            # Check if move is within board boundaries
            if (0 <= move_x < self.width and
                    0 <= move_y < self.height):
                # Check if destination is not occupied
                if self.board[move_y][move_x] is None:
                    moves.append((move_x, move_y))

        return moves



    def setup_initial_board(self):
        """
        Set up initial board with units at the exact map edges
        """
        # Clear existing units
        self.marines.clear()
        self.zerglings.clear()
        self.board = [[None for _ in range(self.width)] for _ in range(self.height)]

        # Calculate vertical center of the board
        center_y = self.height // 2

        # Calculate front and back row counts
        zergling_front_count = self.zergling_count // 2
        zergling_back_count = self.zergling_count - zergling_front_count

        marine_front_count = self.marine_count // 2
        marine_back_count = self.marine_count - marine_front_count

        # Zerglings formation (left edge)
        front_zergling_x = 0  # Leftmost edge of the map
        back_zergling_x = 1  # One step from the left edge

        # Marines formation (right edge)
        front_marine_x = self.width - 1  # Rightmost edge of the map
        back_marine_x = self.width - 2  # One step from the right edge

        # Vertical positioning for Zerglings
        zergling_front_rows = [
            y for y in range(
                center_y - zergling_front_count // 2,
                center_y + (zergling_front_count + 1) // 2
            )
        ]
        zergling_back_rows = [
            y for y in range(
                center_y - zergling_back_count // 2,
                center_y + (zergling_back_count + 1) // 2
            )
        ]

        # Vertical positioning for Marines
        marine_front_rows = [
            y for y in range(
                center_y - marine_front_count // 2,
                center_y + (marine_front_count + 1) // 2
            )
        ]
        marine_back_rows = [
            y for y in range(
                center_y - marine_back_count // 2,
                center_y + (marine_back_count + 1) // 2
            )
        ]

        # Place Zerglings in front rows
        for y in zergling_front_rows:
            if len(self.zerglings) < self.zergling_count:
                zergling = Zergling(front_zergling_x, y)
                self.zerglings.append(zergling)
                self.board[y][front_zergling_x] = zergling

        # Place Zerglings in back rows
        for y in zergling_back_rows:
            if len(self.zerglings) < self.zergling_count:
                zergling = Zergling(back_zergling_x, y)
                self.zerglings.append(zergling)
                self.board[y][back_zergling_x] = zergling

        # Place Marines in front rows
        for y in marine_front_rows:
            if len(self.marines) < self.marine_count:
                marine = Marine(front_marine_x, y)
                self.marines.append(marine)
                self.board[y][front_marine_x] = marine

        # Place Marines in back rows
        for y in marine_back_rows:
            if len(self.marines) < self.marine_count:
                marine = Marine(back_marine_x, y)
                self.marines.append(marine)
                self.board[y][back_marine_x] = marine

        # Verify unit counts
        assert len(
            self.zerglings) == self.zergling_count, f"Zergling count is {len(self.zerglings)}, should be {self.zergling_count}"
        assert len(self.marines) == self.marine_count, f"Marine count is {len(self.marines)}, should be {self.marine_count}"


    def move_unit(self, unit, target):
        # Remove unit from current position
        self.board[unit.y][unit.x] = None

        # Update unit's coordinates
        new_x, new_y = target
        unit.x = new_x
        unit.y = new_y

        # Place unit in new position
        self.board[new_y][new_x] = unit


    def attack_unit(self, attacker, target_unit):
        # Calculate damage based on attacker's characteristics
        damage = attacker.attack

        # Reduce target's health
        target_unit.hp -= damage

        # Check if target is destroyed
        if target_unit.hp <= 0:
            # Remove target from board and unit list
            self.board[target_unit.y][target_unit.x] = None

            if target_unit.team == 'Marine':
                self.marines.remove(target_unit)
            else:
                self.zerglings.remove(target_unit)

    def display_board(self):
        """
        Visualize the current board state with precise unit placement
        """
        # Create a copy of the board to visualize
        display_grid = [[' .' for _ in range(self.width)] for _ in range(self.height)]

        # Print the board with detailed unit placement
        print("\nDebug - Unit Positions:")
        for zergling in self.zerglings:
            print(f"Zergling: x={zergling.x}, y={zergling.y}")
            display_grid[zergling.y][zergling.x] = ' Z'

        for marine in self.marines:
            print(f"Marine: x={marine.x}, y={marine.y}")
            display_grid[marine.y][marine.x] = ' M'

        # Print the board
        for row in display_grid:
            print(''.join(row))

        # Print additional unit information
        print(f"\nMarines: {len(self.marines)}, Zerglings: {len(self.zerglings)}")


    def state_of_board(self):
        # Check if all Marines are dead
        if len(self.marines) == 0:
            return "Zergling"

        # Check if all Zerglings are dead
        if len(self.zerglings) == 0:
            return "Marine"

        # Check if board is full or max turns reached
        if (len(self.marines) + len(self.zerglings) >= self.width * self.height or
                len(self.marines) == 0 or
                len(self.zerglings) == 0):
            return "Draw"

        # Game is still ongoing
        return "U"  # Undetermined/Unfinished

In [25]:
class Unit:
    def __init__(self, x, y, hp, speed, damage, attack_range, team):
        self.x = x
        self.y = y
        self.hp = hp
        self.speed = speed
        self.damage = damage
        self.attack_range = attack_range
        self.team = team  # 'Marine' or 'Zerg'
        self.engaged_in_melee = False



class Marine(Unit):
    def __init__(self, x, y):
        # Using ground DPS as damage (9.8)
        super().__init__(x, y, hp=45, speed=3, damage=9.8, attack_range=5, team='Marine')


class Zergling(Unit):
    def __init__(self, x, y):
        # Using ground DPS as damage (10)
        super().__init__(x, y, hp=35, speed=4, damage=10, attack_range=1, team='Zergling')



In [26]:
class SearchTreeNode:
    def __init__(self, current_board, playing_team, depth=0, alpha=float('-inf'), beta=float('inf')):
        self.current_board = current_board
        self.playing_team = playing_team
        self.depth = depth
        self.children = []
        self.value = None
        self.alpha = alpha
        self.beta = beta

    def is_game_over(self):
        # Check if game is over based on board state
        game_state = self.current_board.state_of_board()
        return game_state != "U"

    def evaluate_position(self):
        # Comprehensive board evaluation
        if self.is_game_over():
            # Terminal state evaluation
            game_state = self.current_board.state_of_board()
            if game_state == "Marine Wins":
                return float('inf')
            elif game_state == "Zergling Wins":
                return float('-inf')
            else:
                return 0

        # Multifactor evaluation
        def calculate_unit_score(units):
            return sum([
                unit.hp +  # Health weight
                unit.damage * 0.5 +  # Damage potential
                (1 / (abs(unit.x - self.current_board.center_x) +
                      abs(unit.y - self.current_board.center_y))) * 10  # Positioning bonus
                for unit in units
            ])

        marine_score = calculate_unit_score(self.current_board.marines)
        zergling_score = calculate_unit_score(self.current_board.zerglings)

        # Positive score favors Marines, negative favors Zerglings
        return marine_score - zergling_score

    def generate_children(self):
        # Generate possible moves for each unit
        self.children = []

        # Determine which units to move based on playing team
        units = (self.current_board.marines if self.playing_team == 'Marine'
                 else self.current_board.zerglings)

        for unit in units:
            # Get possible actions for this unit
            possible_actions = self.current_board.get_possible_actions(unit)

            for action in possible_actions:
                # Create a new board state for each possible action
                new_board = self.create_board_from_action(unit, action)

                # Create child node with opposite team
                child_team = 'Zergling' if self.playing_team == 'Marine' else 'Marine'
                child = SearchTreeNode(
                    new_board,
                    child_team,
                    depth=self.depth + 1,
                    alpha=self.alpha,
                    beta=self.beta
                )

                self.children.append(child)

    def create_board_from_action(self, unit, action):
        # Deep copy of current board
        new_board = copy.deepcopy(self.current_board)

        # Apply the action
        action_type, target = action

        if action_type == 'move':
            # Move unit to new position
            new_board.move_unit(unit, target)
        elif action_type == 'attack':
            # Perform attack
            new_board.attack_unit(unit, target)

        return new_board

    def min_max_value(self, depth, is_maximizing):
        # Base case: reached depth limit or game over
        if depth == 0 or self.is_game_over():
            return self.evaluate_position()

        # Ensure children are generated
        if not self.children:
            self.generate_children()

        # Alpha-Beta Pruning implementation
        if is_maximizing:
            max_eval = float('-inf')
            for child in self.children:
                eval = child.min_max_value(depth - 1, False)
                max_eval = max(max_eval, eval)

                # Alpha-Beta Pruning
                self.alpha = max(self.alpha, eval)
                if self.beta <= self.alpha:
                    break  # Beta cut-off
            return max_eval
        else:
            min_eval = float('inf')
            for child in self.children:
                eval = child.min_max_value(depth - 1, True)
                min_eval = min(min_eval, eval)

                # Alpha-Beta Pruning
                self.beta = min(self.beta, eval)
                if self.beta <= self.alpha:
                    break  # Alpha cut-off
            return min_eval

    def get_best_move_min_max(self, depth=3):
        # Ensure children are generated
        if not self.children:
            self.generate_children()

        best_value = float('-inf')
        best_move = None

        for child in self.children:
            # Evaluate from the opponent's minimizing perspective
            move_value = child.min_max_value(depth, False)

            if move_value > best_value:
                best_value = move_value
                best_move = child

        return best_move


In [27]:
# Create a board
board = CurrentBoard(width=30, height=10,zergling_count=8)

# Initial setup will be automatic by calling setup_initial_board()
board.setup_initial_board()

# Display the initial board state
board.display_board()

# was having issues with the board set up, display and debug helped to show this better.



Debug - Unit Positions:
Zergling: x=0, y=3
Zergling: x=0, y=4
Zergling: x=0, y=5
Zergling: x=0, y=6
Zergling: x=1, y=3
Zergling: x=1, y=4
Zergling: x=1, y=5
Zergling: x=1, y=6
Marine: x=29, y=4
Marine: x=29, y=5
Marine: x=28, y=4
Marine: x=28, y=5
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . M M
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . M M
 Z Z . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Marines: 4, Zerglings: 8


In [28]:
search_tree = SearchTreeNode(board, 'Marine')
best_move = search_tree.get_best_move_min_max(depth=3)

In [31]:
best_move

In [32]:
best_move.current_board.display_board()


Debug - Unit Positions:
Zergling: x=4, y=7
Zergling: x=4, y=0
Zergling: x=4, y=1
Zergling: x=4, y=2
Zergling: x=5, y=7
Zergling: x=5, y=0
Zergling: x=5, y=1
Zergling: x=5, y=2
Marine: x=26, y=1
Marine: x=26, y=2
Marine: x=25, y=1
Marine: x=25, y=5
 . . . . Z Z . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . Z Z . . . . . . . . . . . . . . . . . . . M M . . .
 . . . . Z Z . . . . . . . . . . . . . . . . . . . . M . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . M . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . Z Z . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Marines: 4, Zerglings: 8


In [29]:
def play_StarCraft(width=30, height=10, marine_count=4, zergling_count=8):
    # Ask player to choose starting side
    while True:
        try:
            player_side = input("Choose your side (M for Marine, Z for Zerg): ").upper().strip()

            if player_side == 'M':
                player_side = 'Marine'
                break
            elif player_side == 'Z':
                player_side = 'Zerg'
                break
            else:
                print("Invalid input. Please enter 'M' for Marine or 'Z' for Zerg.")
        except Exception as e:
            print(f"An error occurred: {e}")
            print("Please try again.")

    # Create initial board with specified parameters
    cb = CurrentBoard(width, height, marine_count, zergling_count)
    cb.setup_initial_board()

    # Determine who starts based on player's choice
    if player_side == 'Marine':
        current_player = 'Zergling'  # AI will start with opposite side
        players_turn = False
    else:
        current_player = 'Marine'
        players_turn = True

    # Game loop
    for _ in range(100):  # Prevent infinite loop
        # Display current board state
        cb.display_board()

        # Prompt to continue
        input(f"Press Enter to continue {current_player}'s turn...")

        if players_turn:
            # Player's turn
            search_tree = SearchTreeNode(cb, player_side)
            best_move = search_tree.get_best_move_min_max(depth=3)
            cb = best_move.current_board
        else:
            # AI's turn
            search_tree = SearchTreeNode(cb, current_player)
            best_move = search_tree.get_best_move_min_max(depth=3)
            cb = best_move.current_board

        # Check game state
        game_state = cb.state_of_board()
        if game_state != "U":
            cb.display()
            if game_state == "D":
                print("Draw!")
            else:
                print(f"{game_state} wins!")
            break

        # Switch players
        current_player = 'Zergling' if current_player == 'Marine' else 'Marine'
        players_turn = not players_turn

    print("Game Over")


In [30]:
play_StarCraft()

KeyboardInterrupt: Interrupted by user